In [2]:
import timm
import torch
import torch.nn as nn
import sys
import os
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import datasets, transforms
from timm import create_model
sys.path.append(os.path.abspath(".."))
from data.ImageDataset import ImageDataset

In [3]:
model = create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

/home/ec2-user/CS230Project/venv/lib64/python3.9/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


cuda


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),           
    transforms.ToTensor(),                    
    transforms.Normalize(                     
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

train_dataset = ImageDataset(
    annotations_path="/home/ec2-user/CS230Project/data/annotations/train.json",
    images_dir="/home/ec2-user/CS230Project/data/train",
    transform=transform,
)

val_dataset = ImageDataset(
    annotations_path="/home/ec2-user/CS230Project/data/annotations/val.json",
    images_dir="/home/ec2-user/CS230Project/data/val",
    transform=transform,
)


train_loader = DataLoader(train_dataset, batch_size=32, num_workers=7, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=7, shuffle=False)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device).float()
        labels = labels.view(-1, 1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).float()
            labels = labels.view(-1, 1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            predictions = (outputs > 0.5).float()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    val_accuracy = 100. * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_accuracy:.2f}%")

    scheduler.step()

    checkpoint_path = f"/home/ec2-user/CS230Project/code/models/saved-weights/SwinTransformer/Swin_{epoch+1}.pth"
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Model saved to {checkpoint_path}")


Training Epoch 1/10: 100%|██████████| 1379/1379 [10:07<00:00,  2.27it/s]

Epoch 1, Train Loss: 0.5442, Accuracy: 69.08%


Validation Loss: 0.3609, Accuracy: 85.65%
Model saved to /home/ec2-user/CS230Project/code/models/saved-weights/SwinTransformer/Swin_1.pth


Training Epoch 2/10: 100%|██████████| 1379/1379 [10:50<00:00,  2.12it/s]

Epoch 2, Train Loss: 0.3634, Accuracy: 83.34%


...

Validation Loss: 0.6288, Accuracy: 89.00%
Model saved to /home/ec2-user/CS230Project/code/models/saved-weights/SwinTransformer/Swin_9.pth


Training Epoch 10/10: 100%|██████████| 1379/1379 [07:39<00:00,  3.00it/s]

Epoch 10, Train Loss: 0.0283, Accuracy: 98.80%


Validation Loss: 0.7768, Accuracy: 88.55%
Model saved to /home/ec2-user/CS230Project/code/models/saved-weights/SwinTransformer/Swin_10.pth


: 